# Chapter 01: TensorFlow 소개 및 생태계

## 학습 목표
- TensorFlow 2.x의 핵심 철학과 Eager Execution 이해
- TF 1.x의 정적 그래프 방식과의 차이점 파악
- 현재 환경(버전, 장치, GPU/Metal 백엔드) 확인 방법 습득
- TensorFlow 생태계 전체 조망

## 목차
1. [환경 확인](#환경-확인)
2. [TF 1.x vs TF 2.x: 철학의 변화](#tf-1x-vs-tf-2x)
3. [Eager Execution 이해](#eager-execution)
4. [Apple Silicon & Metal 백엔드](#apple-silicon)
5. [TensorFlow 생태계](#생태계)
6. [요약](#요약)

In [ ]:
# ── 환경 확인 ──────────────────────────────────────────────────
import tensorflow as tf
import numpy as np
import sys

print("=" * 50)
print(f"Python 버전     : {sys.version.split()[0]}")
print(f"TensorFlow 버전 : {tf.__version__}")
print(f"NumPy 버전      : {np.__version__}")
print("=" * 50)

# 사용 가능한 물리적 장치 목록 출력
physical_devices = tf.config.list_physical_devices()
print("\n사용 가능한 장치:")
for device in physical_devices:
    print(f"  - {device.device_type}: {device.name}")

# GPU 전용 목록
gpus = tf.config.list_physical_devices('GPU')
print(f"\nGPU 개수: {len(gpus)}")
if gpus:
    for gpu in gpus:
        print(f"  GPU 장치: {gpu.name}")
else:
    print("  GPU를 찾을 수 없습니다. CPU 모드로 실행됩니다.")

## Apple Silicon & Metal 백엔드 확인 <a name='apple-silicon'></a>

Apple M-시리즈 칩(M1/M2/M3/M4)을 사용하는 경우, `tensorflow-metal` 플러그인을 통해  
GPU 가속을 활용할 수 있습니다.

**설치 방법:**
```bash
pip install tensorflow-metal
```

설치 후 장치 목록에 `GPU` 항목이 나타나면 Metal 백엔드가 정상 동작하는 것입니다.

In [11]:
# Apple Silicon Metal 백엔드 확인
import platform

print(f"운영체제  : {platform.system()}")
print(f"아키텍처  : {platform.machine()}")
print(f"프로세서  : {platform.processor()}")

# Metal GPU 감지
metal_gpus = tf.config.list_physical_devices('GPU')
if metal_gpus and platform.machine() == 'arm64':
    print("\n Apple Silicon + Metal 백엔드 감지됨!")
    print(f"  Metal GPU 장치: {metal_gpus[0].name}")
elif platform.machine() == 'arm64':
    print("\n Apple Silicon 감지 (Metal 미설치 또는 CPU 전용 빌드)")
    print("  'pip install tensorflow-metal' 로 GPU 가속을 활성화하세요.")
else:
    print("\n Intel/AMD 시스템")

운영체제  : Darwin
아키텍처  : arm64
프로세서  : arm

 Apple Silicon 감지 (Metal 미설치 또는 CPU 전용 빌드)
  'pip install tensorflow-metal' 로 GPU 가속을 활성화하세요.


## TF 1.x vs TF 2.x: 철학의 변화 <a name='tf-1x-vs-tf-2x'></a>

### TensorFlow 1.x: 정적 계산 그래프 (Define-and-Run)

TF 1.x에서는 코드를 실행하기 전에 **계산 그래프(Computational Graph)**를 먼저 선언하고,  
이후 `Session`을 통해 그래프를 실행하는 방식이었습니다.

```python
# TF 1.x 방식 (현재는 사용하지 않음)
import tensorflow as tf
a = tf.placeholder(tf.float32)      # 자리 표시자 선언
b = tf.placeholder(tf.float32)
c = tf.add(a, b)                    # 그래프 노드 연결
with tf.Session() as sess:          # 세션에서만 실제 계산
    result = sess.run(c, feed_dict={a: 3.0, b: 4.0})
    print(result)  # 7.0
```

**단점:** 직관적이지 않음, 디버깅 어려움, Python 코드와의 자연스러운 통합 어려움

---

### TensorFlow 2.x: Eager Execution (Define-by-Run)

TF 2.x는 **즉시 실행(Eager Execution)**을 기본으로 채택했습니다.  
Python 코드가 실행되는 즉시 연산이 수행되어, NumPy처럼 직관적으로 사용할 수 있습니다.

| 구분 | TF 1.x | TF 2.x |
|------|--------|--------|
| 실행 방식 | 정적 그래프 + Session | Eager Execution (즉시 실행) |
| 디버깅 | 어려움 (그래프 전체 실행 필요) | 쉬움 (Python 디버거 사용 가능) |
| API | 저수준 API 중심 | Keras 고수준 API 통합 |
| 직관성 | 낮음 | 높음 (NumPy 스타일) |
| 성능 최적화 | 그래프 최적화 자동 | `@tf.function` 데코레이터로 선택적 최적화 |

## Eager Execution 이해 <a name='eager-execution'></a>

Eager Execution이란 Python 코드 한 줄 한 줄이 즉시 실행되는 방식입니다.  
마치 Python의 `int`, `float`처럼 Tensor도 즉시 값을 가집니다.

In [ ]:
# Eager Execution 기본 동작 확인
print("Eager Execution 활성화 여부:", tf.executing_eagerly())

# 즉시 실행: Session 없이 바로 값이 계산됨
a = tf.constant(3.0)
b = tf.constant(4.0)
c = a + b

# TF 1.x였다면 c는 Tensor 노드였고, 값을 보려면 sess.run(c)가 필요했음
# TF 2.x에서는 즉시 값을 출력 가능
print(f"a = {a.numpy()}")
print(f"b = {b.numpy()}")
print(f"c = a + b = {c.numpy()}")
print(f"c 의 타입: {type(c)}")
print(f"c 의 dtype: {c.dtype}")

In [ ]:
# Python 제어 흐름과 자연스럽게 통합 (TF 1.x에서는 tf.cond, tf.while_loop 필요)
def fizzbuzz_tf(n):
    """TF 텐서로 FizzBuzz - 일반 Python if/for 사용 가능"""
    result = []
    for i in tf.range(1, n + 1):
        # 일반 Python 조건문 사용 가능 (Eager Execution 덕분)
        val = i.numpy()
        if val % 15 == 0:
            result.append("FizzBuzz")
        elif val % 3 == 0:
            result.append("Fizz")
        elif val % 5 == 0:
            result.append("Buzz")
        else:
            result.append(str(val))
    return result

print("FizzBuzz (1~20):", fizzbuzz_tf(20))

In [ ]:
# @tf.function: 성능이 중요할 때 그래프 모드로 컴파일
# Eager Execution의 편의성 + 그래프 최적화 성능을 모두 얻는 방법

@tf.function  # 이 데코레이터로 함수를 계산 그래프로 컴파일
def compute_graph(x, y):
    return tf.matmul(x, y) + tf.reduce_sum(x)

x = tf.random.normal([100, 100])
y = tf.random.normal([100, 100])

# 첫 호출: 트레이싱(tracing)으로 그래프 생성
result = compute_graph(x, y)
print(f"계산 결과 shape: {result.shape}")
print("@tf.function으로 그래프 최적화 적용됨")

# 간단한 속도 비교
import time

def eager_fn(x, y):
    return tf.matmul(x, y) + tf.reduce_sum(x)

@tf.function
def graph_fn(x, y):
    return tf.matmul(x, y) + tf.reduce_sum(x)

# 워밍업
_ = eager_fn(x, y)
_ = graph_fn(x, y)

N = 100
start = time.time()
for _ in range(N):
    eager_fn(x, y)
eager_time = time.time() - start

start = time.time()
for _ in range(N):
    graph_fn(x, y)
graph_time = time.time() - start

print(f"\nEager 실행 시간 ({N}회): {eager_time:.4f}초")
print(f"Graph 실행 시간 ({N}회): {graph_time:.4f}초")
if graph_time < eager_time:
    print(f"@tf.function이 {eager_time/graph_time:.2f}배 빠름")
else:
    print("이 경우 Eager가 더 빠르거나 비슷함 (단순 연산의 경우)")

## TensorFlow 생태계 <a name='생태계'></a>

TensorFlow는 단순한 딥러닝 프레임워크가 아니라 **엔드-투-엔드 ML 플랫폼**입니다.

```
┌─────────────────────────────────────────────────┐
│              TensorFlow 생태계                   │
├──────────────┬──────────────┬────────────────────┤
│   데이터     │    모델링    │     배포           │
│  tf.data     │  tf.keras    │  TFLite (모바일)   │
│  tf.io       │  tf.nn       │  TF Serving (서버) │
│  TF Datasets │  tf.layers   │  TF.js (웹브라우저)│
├──────────────┴──────────────┴────────────────────┤
│               모니터링 & 도구                    │
│  TensorBoard │ TF Profiler  │ TF Hub (모델 허브) │
└─────────────────────────────────────────────────┘
```

### 주요 구성 요소

| 구성 요소 | 역할 | 주요 사용 시나리오 |
|-----------|------|-------------------|
| **tf.keras** | 고수준 모델 API | 모델 정의, 학습, 평가 |
| **tf.data** | 데이터 파이프라인 | 대용량 데이터 로딩/전처리 |
| **TensorBoard** | 시각화 도구 | 학습 곡선, 모델 구조 시각화 |
| **TFLite** | 경량화/배포 | 모바일, 임베디드 기기 |
| **TF Hub** | 사전학습 모델 허브 | 전이학습(Transfer Learning) |
| **TF Serving** | 모델 서빙 | 프로덕션 API 서버 |

In [ ]:
# TensorFlow 하위 모듈 탐색
print("tf.keras 구성요소 (일부):")
keras_modules = ['layers', 'models', 'optimizers', 'losses', 'metrics', 'callbacks']
for mod in keras_modules:
    print(f"  tf.keras.{mod}")

print("\ntf 핵심 모듈 (일부):")
tf_modules = ['data', 'io', 'math', 'nn', 'linalg', 'image', 'signal']
for mod in tf_modules:
    print(f"  tf.{mod}")

print("\n전체 서브모듈 수:", len([m for m in dir(tf) if not m.startswith('_')]))

In [ ]:
# tf.data 파이프라인 간단 예시
# 실제 학습에서는 대용량 데이터를 효율적으로 처리하는 핵심 도구

# 간단한 데이터셋 생성
dataset = tf.data.Dataset.range(10)  # 0~9 숫자 데이터셋

# 파이프라인 구성: 셔플 → 배치 → 반복
pipeline = (
    dataset
    .shuffle(buffer_size=10, seed=42)   # 무작위 섞기
    .batch(3)                            # 3개씩 묶기
    .prefetch(tf.data.AUTOTUNE)          # 백그라운드 데이터 준비
)

print("tf.data 파이프라인 출력 (배치 크기=3):")
for batch in pipeline:
    print(f"  배치: {batch.numpy()}")

In [ ]:
# Keras 모델 구조 미리보기 (다음 챕터에서 자세히)
# Sequential API로 간단한 신경망 구조 정의

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
], name='mnist_demo_model')

print("Keras 모델 요약:")
model.summary()

## 요약 <a name='요약'></a>

### 핵심 개념 정리

| 개념 | 설명 |
|------|------|
| **Eager Execution** | 코드 실행 즉시 연산 수행. Python처럼 직관적 |
| **@tf.function** | 함수를 계산 그래프로 컴파일해 성능 최적화 |
| **tf.config** | 장치(GPU/CPU) 확인 및 설정 |
| **Metal 백엔드** | Apple Silicon에서 GPU 가속 (tensorflow-metal) |
| **tf.keras** | 고수준 모델링 API (TF 2.x에 완전 통합) |
| **tf.data** | 효율적인 데이터 입력 파이프라인 |

### 다음 챕터 예고: 02. 텐서와 연산

다음 챕터에서는 TensorFlow의 핵심 데이터 구조인 **Tensor**를 심층적으로 학습합니다:
- Tensor의 rank, shape, dtype
- `tf.constant` vs `tf.Variable`
- 행렬 곱 $C_{ij} = \sum_k A_{ik}B_{kj}$
- 브로드캐스팅 메커니즘
- NumPy와의 상호 변환